In [65]:
import pandas as pd
import json
import os

In [66]:
os.chdir(os.path.join(os.path.expanduser("~"),'hypercardio'))


In [67]:
drugs_interactions_df = pd.read_csv(os.path.join('data','final','drugs_interactions.csv'))
general_info_df = pd.read_csv(os.path.join('data','final','general_info.csv'))

In [68]:
data = {
  "patientDrugsList": [
    "Ramipril",
    "Bisoprolol",
    "Torsemide",
    "Rosuvastatin",
    "Spironolactone"
  ],
  "newDrug": "Advil Cold & Sinus"
}

In [69]:
drugs_interactions_df

,drug_name,parent_id,interacting_drug,id,class,interacting_drug_treated_disease
0,abilify,233-109,abametapir topical,4171-0,moderate,NaN
1,abilify,233-109,abarelix,5-0,moderate,NaN
2,abilify,233-109,abiraterone,3321-0,moderate,NaN
3,abilify,233-109,acarbose,9-0,moderate,NaN
4,abilify,233-109,acebutolol,10-0,moderate,NaN
...,...,...,...,...,...,...
719596,advil cold & sinus,1312-6931,zoledronic acid,2331-0,moderate,Osteoporotic Fractures
719597,advil cold & sinus,1312-6931,zoledronic acid,2331-0,moderate,Pain
719598,advil cold & sinus,1312-6931,zoledronic acid,2331-0,moderate,Prostatic Neoplasms
719599,advil cold & sinus,1312-6931,zoledronic acid,2331-0,moderate,Recurrence


In [70]:
patient_drugs_list = data["patientDrugsList"]
new_drug_name = data["newDrug"].lower()
patient_drugs_list = [i.lower() for i in patient_drugs_list]

new_drug_info = general_info_df[general_info_df.drug_name == new_drug_name]

schema = {
  "drug": None,
  "substance": None,
  "description": None,
  "suggestedReplacement": [],
  "interactions": []
}
found_data =list (new_drug_info.loc[:,['drug_name','drug_raw_name','description']].drop_duplicates().to_dict(orient='index').values())
if found_data != []:
    found_data = found_data[0]
    schema['drug'] = found_data['drug_name']
    schema['substance'] = found_data['drug_raw_name']
    schema['description'] =found_data['description'] if len(found_data['description']) < 100 else found_data['description'][:100]+ '... Read more'

    data = drugs_interactions_df[
        (drugs_interactions_df.drug_name.isin(patient_drugs_list))
        & (drugs_interactions_df.interacting_drug.isin(found_data['drug_raw_name'].split(' and ')))
    ]
    found_interacting_data = data.loc[:,['drug_name','interacting_drug','class','interacting_drug_treated_disease']].drop_duplicates()
    found_interacting_data = found_interacting_data.groupby('drug_name').apply(lambda x: x.sample(1)).reset_index(drop=True)
    for _, group in found_interacting_data.groupby('drug_name'):
        if group.shape[0] > 0:
            drug = group['drug_name'].values[0]
            substance = group['interacting_drug'].values[0]
            level = group['class'].values[0]
            diseases =  list(group['interacting_drug_treated_disease'].values)
            schema['interactions'].append({
                'drug': drug,
                'level': level,
                'diseases': diseases
            })

             drug_name interacting_drug     class  \
486971        ramipril        ibuprofen  moderate   
543283  spironolactone        ibuprofen  moderate   
572892       torsemide        ibuprofen  moderate   

       interacting_drug_treated_disease  
486971          fever/pain/inflammation  
543283          fever/pain/inflammation  
572892          fever/pain/inflammation  


In [71]:
# drugs_interactions_df.loc[(drugs_interactions_df.drug_name.isin(patient_drugs_list))
#         & (drugs_interactions_df.interacting_drug.isin(found_data['drug_raw_name'].split(' and '))),'interacting_drug_treated_disease'] = 'fever/pain/inflammation'

In [72]:
# drugs_interactions_df.to_csv(os.path.join('data','final','drugs_interactions.csv'),header=True,index=False)

In [73]:
    # data = drugs_interactions_df[
    #     (drugs_interactions_df.drug_name.isin(patient_drugs_list))
    #     & (drugs_interactions_df.interacting_drug.isin(found_data['drug_raw_name'].split(' and ')))
    # ]

In [74]:
    data = drugs_interactions_df[
        (drugs_interactions_df.drug_name.isin(patient_drugs_list))
    ]

In [75]:
data['interacting_drug'].unique()

array(['acetazolamide', 'acetohexamide', 'aldesleukin', 'alfentanil',
       'alfuzosin', 'aliskiren', 'allopurinol', 'alogliptin',
       'alprazolam', 'alteplase', 'aluminum carbonate',
       'aluminum hydroxide', 'amifostine', 'amiloride', 'amisulpride',
       'amitriptyline', 'amlodipine', 'amoxapine', 'angiotensin II',
       'anistreplase', 'apomorphine', 'apraclonidine ophthalmic',
       'aripiprazole', 'asenapine', 'asparaginase erwinia chrysanthemi',
       'asparaginase escherichia coli', 'aspirin', 'aurothioglucose',
       'avanafil', 'azathioprine', 'azilsartan medoxomil', 'baclofen',
       'bedaquiline', 'bendroflumethiazide', 'benzthiazide',
       'betamethasone', 'black cohosh', 'brentuximab', 'brexpiprazole',
       'brimonidine ophthalmic', 'brimonidine topical', 'bromfenac',
       'budesonide', 'bumetanide', 'buprenorphine', 'bupropion',
       'buspirone', 'butabarbital', 'butorphanol', 'cabergoline',
       'calaspargase pegol', 'calcium carbonate', 'canaglif

In [76]:
schema

{'drug': 'advil cold & sinus',
 'substance': 'ibuprofen and pseudoephedrine',
 'description': 'Ibuprofen is a nonsteroidal anti-inflammatory drug (NSAID). Pseudoephedrine is a decongestant.',
 'suggestedReplacement': [],
 'interactions': [{'drug': 'ramipril',
   'level': 'moderate',
   'diseases': ['fever/pain/inflammation']},
  {'drug': 'spironolactone',
   'level': 'moderate',
   'diseases': ['fever/pain/inflammation']},
  {'drug': 'torsemide',
   'level': 'moderate',
   'diseases': ['fever/pain/inflammation']}]}